# RNN (work in progress)


> **why we cannot used ANN or CNN for sequential data ?**

- the text data is not of same size.(each example can have the different lenght).

- and cnn or ann canot deal with the input of diff-diff lenght.

- what if convert the every smaple into same size but the amount of unsessary data that we gonna create is crazy not computer efficient at all.

- and also at the time of inferece if we input have diff length then the zero padding is also not gonna work.(i.e larger than out highest our training sample lenght)

- and one thing in ANN or CNN we pass the data at a same time this can cuase the big probelm. 

- cuase the meaining of sentence cann be change or look liek randome noise caues in text the most imp thing is the squence of the words.

- by design the ann cannot remember anything due it we lose the semantic meaning.

> **How RNN solve this problem ?** 

- rnn can handle the sequential data.(mostly text data).

- rnn has this feature due to it can remember the previous features.

> **RNN-Architecure**

- there are three main arc -> 1. many to one , 2. One to many , 3. many to many.

- lets start our discussion with many to one.

- how rnn solve the ann probelm is withe system like self-feedback loop and time-stamp.

- before passing our data into rnn we need to convert data into tokens(brekaing sentence into small chunks) then need to pass.(sequentail information saved). 



In [11]:
!pip3 install jax

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.0 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 390 kB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 66.7 MB 81 kB/s  eta 0:00:01     |███████████████▉                | 33.1 MB 1.2 MB/s eta 0:00:29     |███████████████████████████▋    | 57.6 MB 762 kB/s eta 0:00:12
     |████████████████████████████████| 65 kB 342 kB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [12]:
import jax.numpy as jnp

In [13]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = jnp.arange(5.0)
print(selu(x))

[0.        1.05      2.1       3.1499999 4.2      ]


In [14]:
from jax import random

key = random.key(1701)
x = random.normal(key, (1_000_000,))
%timeit selu(x).block_until_ready()

2.29 ms ± 728 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
from jax import jit

selu_jit = jit(selu)
_ = selu_jit(x)  # compiles on first call
%timeit selu_jit(x).block_until_ready()

425 µs ± 6.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
class RNNModel(nn.Module):


    def __init__(self, rnn_layer, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        if self.rnn.bidirectional:
            self.num_directions = 2
        else:
            self.num_directions = 1
        self.linear = nn.Linear(
            self.num_directions * self.rnn.hidden_size, self.rnn.input_size)

    # forward

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(), self.rnn.input_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, begin_state


    #begin state

    def begin_state(self, device, batch_size=1):
        tensor = torch.zeros((self.num_directions * self.rnn.num_layers, 
                              batch_size, self.rnn.hidden_size), 
                             device=device)
        if isinstance(self.rnn, nn.LSTM):
            return (tensor, tensor) 
        else:
            return tensor
